In [14]:
#!pip install riotwatcher
import requests as req
from riotwatcher import LolWatcher, ApiError
import pandas as pd

# golbal variables
api_key = 'RGAPI-70c0ca22-262a-47ee-9efd-17514682b3b0'
watcher = LolWatcher(api_key)
my_region = 'kr'
number_of_matches = 5
#SKT = ['티원 탑', 'T1 jg test', 'Hide on bush', 'T1 Gumayusi', 'Yukarlna']

In [15]:
sample_user = watcher.summoner.by_name(my_region, 'Hide on bush')
watcher.league.by_summoner(my_region, sample_user['id'])
print(sample_user)
print(sample_user['puuid'])
matches_by_user = watcher.match.matchlist_by_puuid('Asia', sample_user['puuid'], count= number_of_matches)
watcher.match.by_id('Asia', matches_by_user[0])

{'id': 'b81toKJoVjlKhTag-CwOkB3btYJWNLSYd-R5zSh8GJ3Rtg', 'accountId': '54QhLIexx0EO8bj3kgSoEDkz90da-VwZaYRgjFX3bvcd', 'puuid': 'dUpjqhCQHCBl6EZhyUyEPQJuFfMvQGtuW0ZF9Sj0qd6Uf01-N_dm10tiV8g7nsvPR3RmXjg55x7iRA', 'name': 'Hide on bush', 'profileIconId': 6, 'revisionDate': 1639210662523, 'summonerLevel': 513}
dUpjqhCQHCBl6EZhyUyEPQJuFfMvQGtuW0ZF9Sj0qd6Uf01-N_dm10tiV8g7nsvPR3RmXjg55x7iRA


{'info': {'gameCreation': 1639244121000,
  'gameDuration': 1691,
  'gameEndTimestamp': 1639245834174,
  'gameId': 5620711794,
  'gameMode': 'CLASSIC',
  'gameName': 'teambuilder-match-5620711794',
  'gameStartTimestamp': 1639244142971,
  'gameType': 'MATCHED_GAME',
  'gameVersion': '11.24.413.2485',
  'mapId': 11,
  'participants': [{'assists': 5,
    'baronKills': 0,
    'bountyLevel': 0,
    'champExperience': 13398,
    'champLevel': 15,
    'championId': 126,
    'championName': 'Jayce',
    'championTransform': 0,
    'consumablesPurchased': 3,
    'damageDealtToBuildings': 7019,
    'damageDealtToObjectives': 8520,
    'damageDealtToTurrets': 7019,
    'damageSelfMitigated': 17180,
    'deaths': 8,
    'detectorWardsPlaced': 3,
    'doubleKills': 0,
    'dragonKills': 0,
    'firstBloodAssist': False,
    'firstBloodKill': False,
    'firstTowerAssist': False,
    'firstTowerKill': False,
    'gameEndedInEarlySurrender': False,
    'gameEndedInSurrender': False,
    'goldEarned':

In [21]:
column_names = ['game_duration','kills', 'assists', 'deaths','Champion','perks','goldEarned','goldSpent','totalDamageDealtToChampions','visionScore', 'teamPosition']
all_matches = {}
for match in matches_by_user:
  mdata = watcher.match.by_id('Asia', match)
  single_match = []
  for p in mdata['info']['participants']:
    player_row = {}
    player_row['champion'] = p['championName']
    player_row['kills'] = p['kills']
    player_row['assists'] = p['assists']
    player_row['deaths'] = p['deaths']
    player_row['perks'] = p['perks']
    player_row['goldEarned'] = p['goldEarned']
    player_row['goldSpent'] = p['goldSpent']
    player_row['totalDamageToChampions'] = p['totalDamageDealtToChampions']
    player_row['visionScore'] = p['visionScore']
    player_row['teamPosition'] = p['teamPosition']
    single_match.append(player_row)
  all_matches[match] = pd.DataFrame(single_match)



In [23]:
all_matches[matches_by_user[0]]

,champion,kills,assists,deaths,perks,goldEarned,goldSpent,totalDamageToChampions,visionScore,teamPosition
0,Jayce,4,5,8,"{'statPerks': {'defense': 5003, 'flex': 5008, ...",11955,12275,22183,35,TOP
1,LeeSin,5,5,9,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",10134,10050,14480,23,JUNGLE
2,Leblanc,6,5,9,"{'statPerks': {'defense': 5003, 'flex': 5008, ...",10202,9150,25525,23,MIDDLE
3,Ezreal,4,5,9,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",12936,12383,26506,26,BOTTOM
4,Karma,3,7,10,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",7739,7400,12395,75,UTILITY
5,Gragas,6,10,5,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",11459,8975,18992,27,TOP
6,Blitzcrank,4,14,5,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",8992,8950,10886,79,UTILITY
7,Viktor,11,8,5,"{'statPerks': {'defense': 5003, 'flex': 5008, ...",15838,14900,35211,24,MIDDLE
8,Jhin,11,6,6,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",14792,14375,22949,45,BOTTOM
9,XinZhao,13,10,2,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",13657,12500,23902,41,JUNGLE
